In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

tf.__version__

'2.1.0'

In [3]:
gs = pd.read_csv('./games_season.csv')
gt = pd.read_csv('./games_tourney.csv')
gtp = pd.read_csv('./gt_with_preds.csv', index_col=0)

# Two-output models

## Simple two-output modelel

In [18]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer=Adam(0.1), 
              loss='mean_absolute_error')

In [19]:
gtp_train, gtp_test = train_test_split(gtp,
                                       test_size=804,
                                       random_state=42)

## Fit a model with two outputs

In [20]:
# Fit the model
model.fit(gtp_train[['seed_diff', 'pred']],
          gtp_train[['score_1', 'score_2']],
          verbose=2,
          epochs=200,
          validation_split=0.1,
          batch_size=512)

Train on 3087 samples, validate on 343 samples
Epoch 1/200
3087/3087 - 0s - loss: 70.9715 - val_loss: 70.6871
Epoch 2/200
3087/3087 - 0s - loss: 70.0178 - val_loss: 69.8862
Epoch 3/200
3087/3087 - 0s - loss: 69.1323 - val_loss: 69.1338
Epoch 4/200
3087/3087 - 0s - loss: 68.2054 - val_loss: 68.4003
Epoch 5/200
3087/3087 - 0s - loss: 67.3060 - val_loss: 67.5951
Epoch 6/200
3087/3087 - 0s - loss: 66.4285 - val_loss: 66.7906
Epoch 7/200
3087/3087 - 0s - loss: 65.4801 - val_loss: 66.0071
Epoch 8/200
3087/3087 - 0s - loss: 64.5970 - val_loss: 65.2517
Epoch 9/200
3087/3087 - 0s - loss: 63.8327 - val_loss: 64.5392
Epoch 10/200
3087/3087 - 0s - loss: 63.0583 - val_loss: 63.6623
Epoch 11/200
3087/3087 - 0s - loss: 62.3177 - val_loss: 62.7646
Epoch 12/200
3087/3087 - 0s - loss: 61.5095 - val_loss: 61.9314
Epoch 13/200
3087/3087 - 0s - loss: 60.7401 - val_loss: 61.0531
Epoch 14/200
3087/3087 - 0s - loss: 59.9703 - val_loss: 60.2072
Epoch 15/200
3087/3087 - 0s - loss: 59.2086 - val_loss: 59.3527
Ep

Epoch 129/200
3087/3087 - 0s - loss: 9.5527 - val_loss: 9.6096
Epoch 130/200
3087/3087 - 0s - loss: 9.5096 - val_loss: 9.4766
Epoch 131/200
3087/3087 - 0s - loss: 9.5585 - val_loss: 9.6409
Epoch 132/200
3087/3087 - 0s - loss: 9.4964 - val_loss: 9.4798
Epoch 133/200
3087/3087 - 0s - loss: 9.5000 - val_loss: 9.4806
Epoch 134/200
3087/3087 - 0s - loss: 9.4730 - val_loss: 9.5372
Epoch 135/200
3087/3087 - 0s - loss: 9.4677 - val_loss: 9.4703
Epoch 136/200
3087/3087 - 0s - loss: 9.4714 - val_loss: 9.4789
Epoch 137/200
3087/3087 - 0s - loss: 9.4693 - val_loss: 9.5490
Epoch 138/200
3087/3087 - 0s - loss: 9.4641 - val_loss: 9.4523
Epoch 139/200
3087/3087 - 0s - loss: 9.4865 - val_loss: 9.7107
Epoch 140/200
3087/3087 - 0s - loss: 9.5488 - val_loss: 9.4596
Epoch 141/200
3087/3087 - 0s - loss: 9.4709 - val_loss: 9.4056
Epoch 142/200
3087/3087 - 0s - loss: 9.4723 - val_loss: 9.5621
Epoch 143/200
3087/3087 - 0s - loss: 9.4877 - val_loss: 9.4763
Epoch 144/200
3087/3087 - 0s - loss: 9.5121 - val_loss:

## Inspect the model (I)

In [21]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(gtp_train.mean())

[array([[ 0.3982789 , -0.26336065],
       [ 0.45886707, -0.2672135 ]], dtype=float32), array([70.363205, 70.00916 ], dtype=float32)]
season        2001.399125
team_1        5594.013994
team_2        5616.369971
home             0.000000
seed_diff        0.055977
score_diff      -0.060058
score_1         70.949563
score_2         71.009621
won              0.500583
pred            -0.520903
dtype: float64


Did you notice that both output weights are about ~70? This is because, on average, a team will score about 71 points in the tournament.

## Evaluate the model

In [22]:
# Evaluate the model on the tournament test data
print(model.evaluate(gtp_test[['seed_diff', 'pred']], gtp_test[['score_1', 'score_2']], verbose=0))

9.580990988223707


# Single model for classification and regression

## Classification and regression in one model

In [23]:
# Create an input layer with 2 columns
input_tensor = Input(shape=(2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

## Compile and fit model

In [25]:
# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mae', 'binary_crossentropy'], 
              optimizer=Adam(0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(gtp_train[['seed_diff', 'pred']],
          [gtp_train[['score_diff']], gtp_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=512)

Train on 3430 samples
Epoch 1/10
3430/3430 [==============================] - 1s 182us/sample - loss: 12.2002 - dense_4_loss: 11.5155 - dense_5_loss: 0.6879
Epoch 2/10
3430/3430 [==============================] - 0s 17us/sample - loss: 11.6407 - dense_4_loss: 10.9321 - dense_5_loss: 0.6628
Epoch 3/10
3430/3430 [==============================] - 0s 14us/sample - loss: 11.1067 - dense_4_loss: 10.4690 - dense_5_loss: 0.6203
Epoch 4/10
3430/3430 [==============================] - 0s 15us/sample - loss: 10.6634 - dense_4_loss: 10.0656 - dense_5_loss: 0.5916
Epoch 5/10
3430/3430 [==============================] - 0s 15us/sample - loss: 10.3275 - dense_4_loss: 9.7190 - dense_5_loss: 0.5797
Epoch 6/10
3430/3430 [==============================] - 0s 13us/sample - loss: 10.0795 - dense_4_loss: 9.4993 - dense_5_loss: 0.5719
Epoch 7/10
3430/3430 [==============================] - 0s 16us/sample - loss: 9.9014 - dense_4_loss: 9.3303 - dense_5_loss: 0.5611
Epoch 8/10
3430/3430 [=====================

# Inspect the model (II)

In [27]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(gtp_train.mean())

[array([[ 1.2319022 ],
       [-0.03137206]], dtype=float32), array([[0.13757046]], dtype=float32)]
season        2001.399125
team_1        5594.013994
team_2        5616.369971
home             0.000000
seed_diff        0.055977
score_diff      -0.060058
score_1         70.949563
score_2         71.009621
won              0.500583
pred            -0.520903
dtype: float64


In [28]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.13757046

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.5343384756301519
0.7983002541345398


## Evaluate on new data with two metrics

In [29]:
# Evaluate the model on new data
print(model.evaluate(gtp_test[['seed_diff', 'pred']],
               [gtp_test[['score_diff']], gtp_test[['won']]], verbose=False))

[9.948081135156736, 9.355737, 0.5649692]
